In [1]:
!pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [105]:
ncd_app7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_app7.xlsx')
ncd_att7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_att7.xlsx')
ncd_corlt7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corlt7.xlsx')
ncd_corstd7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstd7.xlsx')
ncd_corstdp7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstdp7.xlsx')
ncd_cort7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cort7.xlsx')
ncd_drv7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_drv7.xlsx')
ncd_fin7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_fin7.xlsx')
ncd_pay7 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_pay7.xlsx')
boa7 = pd.read_excel('/Users/user/Documents/BOA 데이터/boa7.xlsx')
ncd7 = pd.concat([ncd_app7, ncd_att7, ncd_corlt7, ncd_corstd7, ncd_corstdp7, ncd_cort7, ncd_drv7, ncd_fin7, ncd_pay7])
ncd7 = ncd7.reset_index(drop=True)

In [106]:
boa7.loc[boa7['상품명'] == 'NAVER WORKS LITE (Deprecated)', '상품명'] = '웍스 코어 Lite'
boa7.loc[boa7['상품명'] == 'NAVER WORKS BASIC (Deprecated)', '상품명'] = '웍스 코어 Standard'
boa7.loc[boa7['상품명'] == '-', '상품명'] = '웍스 코어 Standard'
boa7.loc[boa7['상품명'] == 'NAVER WORKS PREMIUM (Deprecated)', '상품명'] = '웍스 코어 Standard Plus'

In [107]:
grouped_sums = boa7.groupby(['회원번호', '상품명']).agg({
    '매출금액(전월)': 'sum',
    '이용금액(전월)': 'sum'
}).reset_index()

paid_rows = boa7[boa7['paid user 여부'] == 'paid'].copy()

paid_rows = pd.merge(paid_rows, grouped_sums, on=['회원번호', '상품명'], suffixes=('', '_total'))
paid_rows['매출금액(전월)'] = paid_rows['매출금액(전월)_total']
paid_rows['이용금액(전월)'] = paid_rows['이용금액(전월)_total']
paid_rows.drop(columns=['매출금액(전월)_total', '이용금액(전월)_total'], inplace=True)

unique_rows = boa7.groupby(['회원번호', '상품명']).filter(lambda x: len(x) == 1)

final_df = pd.concat([paid_rows, unique_rows], ignore_index=True)

In [108]:
final_df['매출금액(당월)'] = pd.to_numeric(final_df['매출금액(당월)'], errors='coerce')
final_df['매출금액(전월)'] = pd.to_numeric(final_df['매출금액(전월)'], errors='coerce')
final_df['이용금액(당월)'] = pd.to_numeric(final_df['이용금액(당월)'], errors='coerce')
final_df['이용금액(전월)'] = pd.to_numeric(final_df['이용금액(전월)'], errors='coerce')
final_df['단위사용량'] = pd.to_numeric(final_df['단위사용량'], errors='coerce')

original_columns = final_df.columns

aggregated_df = final_df.groupby(['회원번호', '상품명', 'paid user 여부']).agg({
    '매출금액(당월)': 'sum',
    '매출금액(전월)': 'max',
    '이용금액(당월)': 'sum',
    '이용금액(전월)': 'max',
    '단위사용량': 'sum',
}).reset_index()

non_aggregated_cols = final_df.groupby(['회원번호', '상품명', 'paid user 여부']).first().reset_index()

result_df = pd.merge(aggregated_df, non_aggregated_cols.drop(columns=['매출금액(당월)', '매출금액(전월)', '이용금액(당월)', '이용금액(전월)', '단위사용량']),
                     on=['회원번호', '상품명', 'paid user 여부'])

result_df = result_df.sort_values(by=['회원번호', '상품명', 'paid user 여부']).reset_index(drop=True)

result_df = result_df.reindex(columns=original_columns)
result_df = result_df.drop(['상품명.1'], axis=1)


In [110]:
def adjust_revenue_usage(group):
    basic_row = group[group['상품명'] == 'WORKPLACE Basic (Deprecated)']
    payment_row = group[group['상품명'] == '웍스 결재']
    attendance_row = group[group['상품명'] == '웍스 근태']
    finance_row = group[group['상품명'] == '웍스 재무']
    
    if not basic_row.empty:
        basic_revenue = basic_row['매출금액(전월)'].values[0]
        basic_usage = basic_row['이용금액(전월)'].values[0]
        
        if not payment_row.empty:
            group.loc[payment_row.index, '매출금액(전월)'] += int(basic_revenue * (2 / 3))
            group.loc[payment_row.index, '이용금액(전월)'] += int(basic_usage * (3 / 4))
        
        if not attendance_row.empty:
            group.loc[attendance_row.index, '매출금액(전월)'] += int(basic_revenue * (1 / 6))
            group.loc[attendance_row.index, '이용금액(전월)'] += int(basic_usage * (1 / 8))
        
        if not finance_row.empty:
            group.loc[finance_row.index, '매출금액(전월)'] += int(basic_revenue * (1 / 6))
            group.loc[finance_row.index, '이용금액(전월)'] += int(basic_usage * (1 / 8))
    
    return group[group['상품명'] != 'WORKPLACE Basic (Deprecated)']

boa7 = result_df.groupby('회원번호', group_keys=False).apply(adjust_revenue_usage, include_groups=True).reset_index(drop=True)

/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_31657/2800989622.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boa7 = result_df.groupby('회원번호', group_keys=False).apply(adjust_revenue_usage, include_groups=True).reset_index(drop=True)


In [111]:
boa7['매출금액(전월대비증감)'] = boa7['매출금액(당월)'] - boa7['매출금액(전월)']
boa7['매출금액(전월대비%)'] = boa7['매출금액(당월)']/boa7['매출금액(전월)']
boa7['이용금액(전월대비증감)'] = boa7['이용금액(당월)'] - boa7['이용금액(전월)']
boa7['이용금액(전월대비%)'] = boa7['이용금액(당월)']/boa7['이용금액(전월)']

In [162]:
merged7 = pd.merge(ncd7, boa7, how='left', left_on=['회원번호', '미터링유형코드명'], right_on=['회원번호', '상품명'])
merged7['dayCount'] = merged7.groupby(['회원번호', '미터링유형코드명'])['회원번호'].transform('size')
merged7['paid user 여부'] = merged7['요금'].apply(lambda x : 'trial' if x == 0 else 'paid')
merged7['약정할인금액'] = np.where(
    (merged7['미터링유형코드명'].isin(['웍스 코어 Standard Plus'])) & (merged7['약정여부'] == 'Y'), 
    2000, 
    np.where(
        (merged7['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus'])) & (merged7['약정여부'] == 'Y'), 
        1000,
        np.where(
            (merged7['미터링유형코드명'].isin(['웍스 급여'])) & (merged7['약정여부'] == 'Y'), 
            1500,
            np.where(
                (merged7['미터링유형코드명'].isin(['웍스 결재'])) & (merged7['약정여부'] == 'Y'), 
                1000, 
                np.where(
                    (merged7['미터링유형코드명'].isin(['웍스 근태','웍스 재무'])) & (merged7['약정여부'] == 'Y'), 
                    50,
                    np.where(
                        (merged7['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 코어 Standard Plus','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus','웍스 결재','웍스 근태','웍스 재무','웍스 급여'])) & (merged7['약정여부'] == 'N'), 
                        0, 
                        np.nan
                    )
                )
            )
        )
    )
)
merged7['약정할인 후 요금'] = merged7['요금'] - merged7['약정할인금액']
merged7['약정할인율'] = merged7['약정할인금액']/merged7['요금']
merged7['결합할인 여부'] = 'Y'
merged7.loc[merged7['미터링유형코드명'].isin(['웍스 근태', '웍스 재무', '웍스 급여']), '결합할인 여부'] = 'N'
merged7['결합할인 여부'] = np.where(
    (merged7.duplicated(subset=['회원번호','미터링일시','paid user 여부','약정여부','결합할인 여부'], keep=False)) & (merged7['paid user 여부'] == 'paid') & (merged7['약정여부'] == 'Y') & (merged7['결합할인 여부'] == 'Y'),
    'Y',
    'N')
merged7.loc[merged7['약정여부'] == 'N', '결합할인 여부'] = 'N'
merged7['일매출(당일)'] = np.where(
    (merged7['결합할인 여부'] == 'Y'),
    ((merged7['요금']/31)*merged7['사용량']*(1-merged7['약정할인율']))*0.9,
    (merged7['요금']/31)*merged7['사용량']*(1-merged7['약정할인율']))

In [163]:
merged7 = merged7.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'])
merged7['일매출(전일)'] = merged7.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].shift(1)
merged7['일매출 증가율'] = ((merged7['일매출(당일)'] - merged7['일매출(전일)']) / merged7['일매출(전일)'])
merged7['누적 일매출'] = merged7.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].cumsum()
merged7['월매출(당월)'] = merged7.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].transform('sum')
merged7 = merged7.drop(['dayCount'], axis = 1)

In [164]:
def domain_class(group):
    if ((group == 20240701).any() and (group == 20240731).any()):
        return '기존' 
    elif ((group != 20240701).all() and (group != 20240731).all()):
        return '신규/이탈'
    elif (group != 20240701).all():
        return '신규'
    elif (group != 20240731).all():
        return '이탈'
    else :
        return '기타'


merged7['회원구분'] = merged7.groupby(['회원번호', '미터링유형코드명'])['미터링일시'].transform(lambda x: domain_class(x))


In [165]:
merged7['구분1'] = 'Business Application'
merged7['구분2'] = 'Business Applications'
merged7['구분3'] = merged7['인스턴스유형코드명']
merged7['상품명'] = merged7['미터링유형코드명']
merged7['플랫폼'] = '민간'
merged7['리전명_y'] = '한국'
merged7['리전번호_y'] = 1
merged7 = merged7.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'], ascending = [True, False, True])

In [166]:
merged7 = merged7.head(100000)

In [168]:
merged7

,회원번호,회원그룹코드,회원그룹명,파트너사회원번호,청구유형코드,청구유형코드명,청구유형상세코드,청구유형상세코드명,인스턴스유형코드,인스턴스유형코드명,...,약정할인금액,약정할인 후 요금,약정할인율,결합할인 여부,일매출(당일),일매출(전일),일매출 증가율,누적 일매출,월매출(당월),회원구분
160452,734,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_LITE_USER_CNT,웍스 코어 Lite,WORK2,NAVER WORKS,...,1000.0,3000.0,0.250000,N,3290.322581,NaN,NaN,3290.322581,102870.967742,기존
160453,734,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_LITE_USER_CNT,웍스 코어 Lite,WORK2,NAVER WORKS,...,1000.0,3000.0,0.250000,N,3290.322581,3290.322581,0.0,6580.645161,102870.967742,기존
160454,734,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_LITE_USER_CNT,웍스 코어 Lite,WORK2,NAVER WORKS,...,1000.0,3000.0,0.250000,N,3290.322581,3290.322581,0.0,9870.967742,102870.967742,기존
160455,734,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_LITE_USER_CNT,웍스 코어 Lite,WORK2,NAVER WORKS,...,1000.0,3000.0,0.250000,N,3290.322581,3290.322581,0.0,13161.290323,102870.967742,기존
160456,734,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,CORE_LITE_USER_CNT,웍스 코어 Lite,WORK2,NAVER WORKS,...,1000.0,3000.0,0.250000,N,3290.322581,3290.322581,0.0,16451.612903,102870.967742,기존
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231300,2655594,PATR,파트너회원,2673247.0,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,1000.0,6000.0,0.142857,N,2322.580645,2322.580645,0.0,53419.354839,72000.000000,기존
231301,2655594,PATR,파트너회원,2673247.0,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,1000.0,6000.0,0.142857,N,2322.580645,2322.580645,0.0,55741.935484,72000.000000,기존
231302,2655594,PATR,파트너회원,2673247.0,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,1000.0,6000.0,0.142857,N,2322.580645,2322.580645,0.0,58064.516129,72000.000000,기존
231303,2655594,PATR,파트너회원,2673247.0,WORK2,NAVER WORKS,CORE_STAND_USER_CNT,웍스 코어 Standard,WORK2,NAVER WORKS,...,1000.0,6000.0,0.142857,N,2322.580645,2322.580645,0.0,60387.096774,72000.000000,기존


In [170]:
merged7.to_excel('/Users/user/Documents/업무문서/boaa7.xlsx', index = False)